In [79]:
import os
import pandas as pd
import numpy as np
import csv

from pandas.tseries.holiday import USFederalHolidayCalendar

Get ALL data in single DF

In [2]:
Y1_files = ['RBSAM_Y1_PART 1 OF 4.csv', 'RBSAM_Y1_PART 2 OF 4.csv', 
            'RBSAM_Y1_PART 3 OF 4.csv', 'RBSAM_Y1_PART 4 OF 4.csv']
Y2_files = ['RBSAM_Y2_PART 1 OF 5.csv', 'RBSAM_Y2_PART 2 OF 5.csv', 'RBSAM_Y2_PART 3 OF 5.csv', 
            'RBSAM_Y2_PART 4 OF 5.csv', 'RBSAM_Y2_PART 5 OF 5.csv']
filedir = 'rbsa/raw/'

device_map_file = 'rbsa/device_map.csv'

In [3]:
device_level = True

if device_level:
    device_map = pd.read_csv(device_map_file)

    rbsa_dict = {} # device name to enduse
    enduses = set()
    
    for index, row in device_map.iterrows():
        key = row["enduse_code"]
        enduse = row["eu"]
        units = row["units"]
        rbsa_dict[key] = {"enduse": enduse, "unit": units}
        if rbsa_dict[key]['unit'] == 'kWh':
            enduses.add(rbsa_dict[key]['enduse'])

In [4]:
 master_y1 = pd.DataFrame()

for filename in Y1_files:
    df = pd.read_csv(filedir + filename)
    df['time'] = pd.to_datetime(df['time'], format='%d%b%y:%H:%M:%S')
    df = df.fillna(0)
    df['siteid'] = df['siteid'].astype(str)  
    df = df.groupby('siteid').resample('60T', on='time').sum()
    
    if device_level:
        column_map = {k: [] for k in enduses}

        for column in list(df): 
            col_name = column.split()[0]

            if (col_name == 'Hours') or (column[:10] == 'Fahrenheit'):
                continue  
            if rbsa_dict[col_name]['unit'] == 'kWh':    
                column_map[rbsa_dict[col_name]['enduse']].append(column)

        # remove unused
        column_map.pop('', None)
        column_map.pop('ignore', None)

        for enduse in column_map.keys():
            df[enduse] = df[column_map[enduse]].sum(axis=1)

        df = df[list(column_map.keys())]

    master_y1 = pd.concat([master_y1, df])

In [5]:
master_y1.columns

Index(['Panel', 'Cooling', 'Service', 'InteriorLighting', 'WaterHeating',
       'Total', 'Appliances', 'ExteriorLighting', 'Ventilation', 'Electronics',
       'Refrigeration', 'Miscellaneous', 'Cooking', 'Heating', 'HeatCool'],
      dtype='object')

In [6]:
 master_y2 = pd.DataFrame()

for filename in Y2_files:
    df = pd.read_csv(filedir + filename)
    df['time'] = pd.to_datetime(df['time'], format='%d%b%y:%H:%M:%S')
    df = df.fillna(0)
    df['siteid'] = df['siteid'].astype(str)  
    df = df.groupby('siteid').resample('60T', on='time').sum()
    
    if device_level:
        column_map = {k: [] for k in enduses}

        for column in df.columns:
            try:
                if rbsa_dict[column]['unit'] == 'kWh':    
                    column_map[rbsa_dict[column]['enduse']].append(column)
            except KeyError:
                continue # unused columns

        # remove unused
        column_map.pop('', None)
        column_map.pop('ignore', None)

        for enduse in column_map.keys():
            df[enduse] = df[column_map[enduse]].sum(axis=1)

        df = df[list(column_map.keys())]

    master_y2 = pd.concat([master_y1, df])

In [7]:
master_y2.columns

Index(['Panel', 'Cooling', 'Service', 'InteriorLighting', 'WaterHeating',
       'Total', 'Appliances', 'ExteriorLighting', 'Ventilation', 'Electronics',
       'Refrigeration', 'Miscellaneous', 'Cooking', 'Heating', 'HeatCool'],
      dtype='object')

In [8]:
master_df = pd.concat([master_y1, master_y2])
master_df = master_df.sort_values(by='siteid')
master_df = master_df.reset_index(level=[0,1])
master_df = master_df.groupby('siteid').resample('60T', on='time').sum()

In [9]:
master_df.to_csv('rbsa.csv')

In [30]:
# for site in master_df.index.get_level_values('siteid').unique():
#     site_df = master_df.loc[master_df.index.get_level_values('siteid') == site]
#     filename = 'rbsa_sites/'+str(site)+'.csv'
#     site_df.to_csv(filename)

Seperate into single file for Heat Pump and AC

In [12]:
sites = []

for filename in Y1_files + Y2_files:
    df = pd.read_csv(filedir+filename)
    sites = sites + list(df['siteid'].unique())
    
sites = set(sites)

In [13]:
hvac_df = pd.read_csv('rbsa/2011 RBSA Single Family Database-HVACcooling.csv', index_col=0)

In [30]:
exist_count = 0
hvac_dict = {}

for site in sites:
    if site in hvac_df['siteid'].unique():
        exist_count += 1
        hvac_type = hvac_df.loc[hvac_df['siteid'] == site]['HVACType'].iloc[0]
        if hvac_type in hvac_dict.keys():
            hvac_dict[hvac_type].append(str(site))
        else:
            hvac_dict[hvac_type] = [str(site)]
        
exist_count

61

In [31]:
hvac_dict

{'heatpump': ['20998',
  '13445',
  '10887',
  '24203',
  '11418',
  '22699',
  '14508',
  '12975',
  '14542',
  '21799',
  '14646',
  '24400',
  '23960',
  '14331',
  '21414',
  '13248',
  '23028',
  '20469',
  '11775'],
 'centralAC': ['23049',
  '20020',
  '20553',
  '13903',
  '13400',
  '22615',
  '21615',
  '22138',
  '22144',
  '21678',
  '20685',
  '24808',
  '20753',
  '14102',
  '21298',
  '24378',
  '20807',
  '21343',
  '21905',
  '22938',
  '13261',
  '22514'],
 'windowshaker': ['13336',
  '23618',
  '13912',
  '24684',
  '14073',
  '21355',
  '14329',
  '21499'],
 'PTAC': ['23138', '13974', '22822', '14150', '14277'],
 'dhp': ['21143', '22222', '12507', '24495'],
 'gshp': ['22177'],
 'heatpumpdualfuel': ['14174', '13222']}

In [32]:
heatpump_df = master_df.loc[master_df.index.get_level_values('siteid').isin(hvac_dict['heatpump'])]

In [33]:
ac_df = master_df.loc[master_df.index.get_level_values('siteid').isin(hvac_dict['centralAC'])]

In [130]:
from datetime import timedelta 
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2011-01-01', end='2016-12-31').to_pydatetime()

holiday_allhours = []

for day in holidays:
    for hour in range(24):
        holiday_allhours.append(day+timedelta(hours=hour))
    
# holidays = [d.date() for d in holidays]

In [129]:
# holiday_allhours

Convert to correct format

In [140]:
def create_df(input_df):
    output_df = pd.DataFrame()

    for month in range(1,13):
        month_df = input_df.loc[input_df.index.get_level_values('time').month == month]

        # weekday 
        weekday_df = month_df.loc[month_df.index.get_level_values('time').weekday < 5]   
        for hour in range(24):
            hour_df = weekday_df.loc[weekday_df.index.get_level_values('time').hour == hour]
            hour_df = hour_df.loc[~hour_df.index.get_level_values('time').isin(holidays)]
            hour_df = hour_df.mean(axis=0)
            hour_df['Month'] = int(month)
            hour_df['Daytype'] = 'WEEKDAY'
            hour_df['Hour'] = int(hour)
            output_df = output_df.append(hour_df, ignore_index=True)

        # saturday 
        saturday_df = month_df.loc[month_df.index.get_level_values('time').weekday == 5]
        for hour in range(24):
            hour_df = saturday_df.loc[saturday_df.index.get_level_values('time').hour == hour]
            hour_df = hour_df.loc[~hour_df.index.get_level_values('time').isin(holidays)]
            hour_df = hour_df.mean(axis=0)
            hour_df['Month'] = int(month)
            hour_df['Daytype'] = 'SATURDAY'
            hour_df['Hour'] = int(hour)
            output_df = output_df.append(hour_df, ignore_index=True)

        # sunday 
        sunday_df = month_df.loc[month_df.index.get_level_values('time').weekday == 6]
        for hour in range(24):
            hour_df = sunday_df.loc[sunday_df.index.get_level_values('time').hour == hour]
            hour_df = hour_df.loc[~hour_df.index.get_level_values('time').isin(holidays)]
            hour_df = hour_df.mean(axis=0)
            hour_df['Month'] = int(month)
            hour_df['Daytype'] = 'SUNDAY'
            hour_df['Hour'] = int(hour)
            output_df = output_df.append(hour_df, ignore_index=True)
            
        # holiday
        month_df.index.get_level_values('time') 
        holiday_df = month_df.loc[month_df.index.get_level_values('time').isin(holidays)]
        print(holiday_df)
        for hour in range(24):
            hour_df = holiday_df.loc[holiday_df.index.get_level_values('time').hour == hour]
            hour_df = hour_df.mean(axis=0)
            hour_df['Month'] = int(month)
            hour_df['Daytype'] = 'HOLIDAY'
            hour_df['Hour'] = int(hour)
            output_df = output_df.append(hour_df, ignore_index=True)
            
    output_df = output_df[['Month', 'Daytype', 'Hour', 'Appliances', 'Cooking', 'Cooling', 'Electronics',
                          'ExteriorLighting', 'HeatCool', 'Heating', 'InteriorLighting', 'Miscellaneous', 'Panel', 
                          'Refrigeration', 'Service', 'Total', 'Ventilation', 'WaterHeating']]
            
    return output_df

In [141]:
create_df(heatpump_df).to_csv('rbsa_heatpumps.csv', index=False)

DatetimeIndex(['2013-01-01 00:00:00', '2013-01-01 01:00:00',
               '2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               ...
               '2014-01-31 14:00:00', '2014-01-31 15:00:00',
               '2014-01-31 16:00:00', '2014-01-31 17:00:00',
               '2014-01-31 18:00:00', '2014-01-31 19:00:00',
               '2014-01-31 20:00:00', '2014-01-31 21:00:00',
               '2014-01-31 22:00:00', '2014-01-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=16368, freq=None)
                   Panel  Cooling    Service  InteriorLighting  WaterHeating  \
siteid time                                                                    
10887  2013-01-01   0.00      0.0  29.740000          0.088606      0.000000   
       2013-01-21   0.00      0.0   8.860000          

DatetimeIndex(['2013-02-01 00:00:00', '2013-02-01 01:00:00',
               '2013-02-01 02:00:00', '2013-02-01 03:00:00',
               '2013-02-01 04:00:00', '2013-02-01 05:00:00',
               '2013-02-01 06:00:00', '2013-02-01 07:00:00',
               '2013-02-01 08:00:00', '2013-02-01 09:00:00',
               ...
               '2014-02-28 14:00:00', '2014-02-28 15:00:00',
               '2014-02-28 16:00:00', '2014-02-28 17:00:00',
               '2014-02-28 18:00:00', '2014-02-28 19:00:00',
               '2014-02-28 20:00:00', '2014-02-28 21:00:00',
               '2014-02-28 22:00:00', '2014-02-28 23:00:00'],
              dtype='datetime64[ns]', name='time', length=14784, freq=None)
                   Panel  Cooling    Service  InteriorLighting  WaterHeating  \
siteid time                                                                    
10887  2013-02-18   0.00      0.0  11.020000          0.000000          0.00   
11418  2013-02-18   0.00      0.0   0.620000          

DatetimeIndex(['2012-05-01 00:00:00', '2012-05-01 01:00:00',
               '2012-05-01 02:00:00', '2012-05-01 03:00:00',
               '2012-05-01 04:00:00', '2012-05-01 05:00:00',
               '2012-05-01 06:00:00', '2012-05-01 07:00:00',
               '2012-05-01 08:00:00', '2012-05-01 09:00:00',
               ...
               '2014-05-31 14:00:00', '2014-05-31 15:00:00',
               '2014-05-31 16:00:00', '2014-05-31 17:00:00',
               '2014-05-31 18:00:00', '2014-05-31 19:00:00',
               '2014-05-31 20:00:00', '2014-05-31 21:00:00',
               '2014-05-31 22:00:00', '2014-05-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=19520, freq=None)
                   Panel  Cooling    Service  InteriorLighting  WaterHeating  \
siteid time                                                                    
10887  2012-05-28    0.0      0.0   2.860000          0.000000          0.02   
11418  2012-05-28    0.0      0.0   0.840000          

DatetimeIndex(['2012-06-01 00:00:00', '2012-06-01 01:00:00',
               '2012-06-01 02:00:00', '2012-06-01 03:00:00',
               '2012-06-01 04:00:00', '2012-06-01 05:00:00',
               '2012-06-01 06:00:00', '2012-06-01 07:00:00',
               '2012-06-01 08:00:00', '2012-06-01 09:00:00',
               ...
               '2014-06-30 14:00:00', '2014-06-30 15:00:00',
               '2014-06-30 16:00:00', '2014-06-30 17:00:00',
               '2014-06-30 18:00:00', '2014-06-30 19:00:00',
               '2014-06-30 20:00:00', '2014-06-30 21:00:00',
               '2014-06-30 22:00:00', '2014-06-30 23:00:00'],
              dtype='datetime64[ns]', name='time', length=18346, freq=None)
Empty DataFrame
Columns: [Panel, Cooling, Service, InteriorLighting, WaterHeating, Total, Appliances, ExteriorLighting, Ventilation, Electronics, Refrigeration, Miscellaneous, Cooking, Heating, HeatCool]
Index: []
DatetimeIndex(['2012-07-01 00:00:00', '2012-07-01 01:00:00',
               '201

DatetimeIndex(['2012-08-01 00:00:00', '2012-08-01 01:00:00',
               '2012-08-01 02:00:00', '2012-08-01 03:00:00',
               '2012-08-01 04:00:00', '2012-08-01 05:00:00',
               '2012-08-01 06:00:00', '2012-08-01 07:00:00',
               '2012-08-01 08:00:00', '2012-08-01 09:00:00',
               ...
               '2013-08-31 14:00:00', '2013-08-31 15:00:00',
               '2013-08-31 16:00:00', '2013-08-31 17:00:00',
               '2013-08-31 18:00:00', '2013-08-31 19:00:00',
               '2013-08-31 20:00:00', '2013-08-31 21:00:00',
               '2013-08-31 22:00:00', '2013-08-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=16368, freq=None)
Empty DataFrame
Columns: [Panel, Cooling, Service, InteriorLighting, WaterHeating, Total, Appliances, ExteriorLighting, Ventilation, Electronics, Refrigeration, Miscellaneous, Cooking, Heating, HeatCool]
Index: []
DatetimeIndex(['2012-09-01 00:00:00', '2012-09-01 01:00:00',
               '201

DatetimeIndex(['2012-11-01 00:00:00', '2012-11-01 01:00:00',
               '2012-11-01 02:00:00', '2012-11-01 03:00:00',
               '2012-11-01 04:00:00', '2012-11-01 05:00:00',
               '2012-11-01 06:00:00', '2012-11-01 07:00:00',
               '2012-11-01 08:00:00', '2012-11-01 09:00:00',
               ...
               '2013-11-30 14:00:00', '2013-11-30 15:00:00',
               '2013-11-30 16:00:00', '2013-11-30 17:00:00',
               '2013-11-30 18:00:00', '2013-11-30 19:00:00',
               '2013-11-30 20:00:00', '2013-11-30 21:00:00',
               '2013-11-30 22:00:00', '2013-11-30 23:00:00'],
              dtype='datetime64[ns]', name='time', length=15840, freq=None)
                   Panel  Cooling    Service  InteriorLighting  WaterHeating  \
siteid time                                                                    
10887  2012-11-12   0.00      0.0   6.000000          0.000000      0.000000   
       2012-11-22   0.00      0.0   7.380000          

DatetimeIndex(['2012-12-01 00:00:00', '2012-12-01 01:00:00',
               '2012-12-01 02:00:00', '2012-12-01 03:00:00',
               '2012-12-01 04:00:00', '2012-12-01 05:00:00',
               '2012-12-01 06:00:00', '2012-12-01 07:00:00',
               '2012-12-01 08:00:00', '2012-12-01 09:00:00',
               ...
               '2013-12-31 14:00:00', '2013-12-31 15:00:00',
               '2013-12-31 16:00:00', '2013-12-31 17:00:00',
               '2013-12-31 18:00:00', '2013-12-31 19:00:00',
               '2013-12-31 20:00:00', '2013-12-31 21:00:00',
               '2013-12-31 22:00:00', '2013-12-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=16368, freq=None)
                   Panel  Cooling    Service  InteriorLighting  WaterHeating  \
siteid time                                                                    
10887  2012-12-25   0.00      0.0  15.180000          0.114500      0.000000   
11418  2012-12-25   0.00      0.0   0.420000          

In [109]:
create_df(ac_df).to_csv('rbsa_ACs.csv', index=False)